In [54]:
#!pip install ortools
!pip install dtale

     |████████████████████████████████| 7.6MB 2.9MB/s 
     |████████████████████████████████| 1.8MB 50.6MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
     |████████████████████████████████| 645kB 45.0MB/s 
     |████████████████████████████████| 71kB 9.3MB/s 
     |████████████████████████████████| 112kB 48.0MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 358kB 43.7MB/s 
     |████████████████████████████████| 1.2MB 46.1MB/s 
     |████████████████████████████████| 3.4MB 41.9MB/s 
     |████████████████████████████████| 194kB 43.4MB/s 
     |████████████████████████████████| 1.8MB 41.4MB/s 
  Created wheel for dash-bootstrap-components: filename=dash_bootstrap_components-0.10.5-cp36-none-any.whl size=180066 sha256=12bf9f560a7398acdcc3cb858b46e33cb7a009cc432f77757a1bb644d4b6bbd5
  Stored in directory: /root/.cache/pip/wheels/41/7a/35/3c7ab615003bb9db

In [2]:
import pandas as pd
import numpy as np
import collections
#from ortools.sat.python import cp_model
#from dtale import show
#import dtale.app as dtale_app
#dtale_app.USE_COLAB = True
# Clone the entire repo.
!git clone -l -s https://github.com/mswastik/optimization.git cloned-repo
%cd cloned-repo

Cloning into 'cloned-repo'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 80 (delta 20), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (80/80), done.
/content/cloned-repo


In [74]:
df=pd.read_excel('orders.xlsx')
ma=pd.read_excel('item master.xlsx')
bu=pd.read_excel('item master.xlsx',sheet_name='bulk')
pa=pd.read_excel('item master.xlsx',sheet_name='packing')

In [75]:
df = df[df['Line']=='Tmmthpkhti Limi']
df = df[df['Plant']==1024]
del df['Line']
df['due'] = df['Dispatch Date'] - df['SO Date'].min()
df['due'] = df['due'].dt.days
df['sodate'] = df['SO Date'] - df['SO Date'].min()
df['sodate'] = df['sodate'].dt.days
df.reset_index(inplace=True,drop=True)
df['Key']=df['SO'].astype('str')+df['FG Code']
del df['Dispatch Date']
del df['SO Date']
del df['Description']
del df['Customer']

In [112]:
df[df["Key"]=='17023287gd29907800iX']['Order Qty'].values[0]

216

In [82]:
df

,SO,FG Code,Order Qty,Plant,due,sodate,Key
0,17023287,gd29904300iX,486,1024,223,146,17023287gd29904300iX
1,17023287,gd29907800iX,216,1024,223,146,17023287gd29907800iX
2,17023287,gd39000800iX,360,1024,223,146,17023287gd39000800iX
3,17023287,gd55102400iX,180,1024,223,146,17023287gd55102400iX
4,17023572,gd43208500iX,756,1024,225,170,17023572gd43208500iX
...,...,...,...,...,...,...,...
179,17023468,gd28007800iX,336,1024,307,246,17023468gd28007800iX
180,17023468,gd28001700iX,640,1024,307,246,17023468gd28001700iX
181,17023833,gd55001700iX,320,1024,307,245,17023833gd55001700iX
182,17023833,gd55102400iX,360,1024,307,245,17023833gd55102400iX


In [123]:
df['Order Qty'] = df[df['Order Qty']!=0]['Order Qty']
df['Order Qty']=df['Order Qty'].fillna(0).astype('int')

In [79]:
jobs =df['Key']
op =[1,2,3,4]
mc = [1,2,3,4,5,6]
horizon = max(df["due"])+100
due = df.set_index('Key')
dur=df[['Key','FG Code','Order Qty']]
dur=dur.merge(pa[["FG Code","Line","Output"]], on="FG Code")
dur["duration"]=dur["Order Qty"]/dur["Output"]
#del dur["Order Qty"]
dur.set_index(['Key','Line'],inplace=True)
del dur['FG Code']
#del dur['Output']

In [83]:
dur['duration']=dur['duration'].fillna(0).astype('int')
dur['Output']=dur['Output'].fillna(0).astype('int')

In [ ]:
dur['duration'].loc[('17023287gd29904300iX',1)]

21

In [55]:
from dtale import show
import dtale.app as dtale_app
dtale_app.USE_COLAB = True
show(df)

https://n6ekujl3anh-496ff2e9c6d22116-40000-colab.googleusercontent.com/dtale/main/1

In [7]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()

In [84]:
task_type = collections.namedtuple('task_type', 'start end duration')
all_tasks = {}
#sch = dict.fromkeys([1,2,3,4,5,6],[])
delay = model.NewIntVar(0, horizon, 'delay')
for j in jobs:
    for m in mc:
        suffix = '_%s_%i' % (j,m)
        start = model.NewIntVar(0, horizon, "start"+suffix)
        end = model.NewIntVar(0, horizon, "end"+suffix)
        duration = model.NewIntervalVar(start, dur['duration'].loc[(j,m)], end, "duration"+suffix)
        #delay += due['due'].loc[j]-end
        #model.Add(due['due'].loc[j]-end)
        #sch[m].append(duration)
        all_tasks[(j, m)] = task_type(start=start, end=end, duration=duration)
    delay += due['due'].loc[j]-end

In [ ]:
for j in jobs:
        model.Add(sch[m])

In [85]:
mtj = {}
for m in mc:
    mjob=[]
    for j in jobs:
        mjob.append(all_tasks[(j, m)].duration)
    mtj[m] = mjob
    model.AddNoOverlap(mjob)
model.Minimize(delay)

In [127]:
for j in df['Key']:
  for m in mc:
    print((all_tasks[(j, m)].end-all_tasks[(j, m)].start)*dur['Output'].loc[(j,m)]
              >= 
              df[df["Key"]==j]['Order Qty'].values[0])
    # Remove 0 from packing speed

(22 * (end_17023287gd29904300iX_1 + -start_17023287gd29904300iX_1)) >= 486
(20 * (end_17023287gd29904300iX_2 + -start_17023287gd29904300iX_2)) >= 486
(22 * (end_17023287gd29904300iX_3 + -start_17023287gd29904300iX_3)) >= 486
(30 * (end_17023287gd29904300iX_4 + -start_17023287gd29904300iX_4)) >= 486
(20 * (end_17023287gd29904300iX_5 + -start_17023287gd29904300iX_5)) >= 486
(20 * (end_17023287gd29904300iX_6 + -start_17023287gd29904300iX_6)) >= 486
(20 * (end_17023287gd29907800iX_1 + -start_17023287gd29907800iX_1)) >= 216
(20 * (end_17023287gd29907800iX_2 + -start_17023287gd29907800iX_2)) >= 216
(20 * (end_17023287gd29907800iX_3 + -start_17023287gd29907800iX_3)) >= 216
False
(20 * (end_17023287gd29907800iX_5 + -start_17023287gd29907800iX_5)) >= 216
(20 * (end_17023287gd29907800iX_6 + -start_17023287gd29907800iX_6)) >= 216
False
False
False
False
False
False
(4 * (end_17023287gd55102400iX_1 + -start_17023287gd55102400iX_1)) >= 180
(4 * (end_17023287gd55102400iX_2 + -start_17023287gd5510240

In [124]:
for j in df['Key']:
  for m in mc:
    model.Add((all_tasks[(j, m)].end-all_tasks[(j, m)].start)*dur['Output'].loc[(j,m)]
              >= 
              df[df["Key"]==j]['Order Qty'].values[0])
    # Remove 0 from packing speed, it is causing this error

TypeError: ignored

In [69]:
solver = cp_model.CpSolver()
solver.Solve(model)
solver.StatusName()

'INFEASIBLE'

**Indices**  
j - job  
m - machine  

**Decision Variable**  
$s_{j,m}$ - start time of job j at machine m  
$e_{j,m}$ - end time of job j at machine m  
$d_{j,m}$ - duration of job j at machine m  

**Parameters**  
H - Horizon  
$d_j$ - delivery date of job j  
$q_j$ - order qty of job j  
$o_{j,m}$ - output per hour of job j at machine m  

**Objective**  
$ \sum_{j} d_j - max(e_{j,m})  \qquad \forall m \in mc$  

**Constraints**  
$ \sum_m e_{j,m} < s_{i,m} or s_{j,m} > e_{i,m} \qquad \forall j,i \in jobs$  
$ \sum_j o_{j,m} \times d_{j,m} >= q_j \qquad \forall m \in ma $
